# Package

In [ ]:
import numpy as np
import pandas as pd
import os
import networkx as nx
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_score
import csv
from tqdm import tqdm

# Hyperparameter (Yes or No)

In [2]:
DO_HYPERPARAMETER_TUNING = True # Default: True

# Load the data

In [3]:
base_path = os.getcwd()
node_info_path = os.path.join(base_path, "node_information.csv")
train_path = os.path.join(base_path, "train.txt")
test_path = os.path.join(base_path, "test.txt")

In [4]:
node_features = {}
with open(node_info_path, "r") as f:
    for line in f:
        values = line.strip().split(',')
        node_id = int(values[0])
        features = np.array([float(x) for x in values[1:]])
        node_features[node_id] = features

In [5]:
# Find the maximum node ID to determine feature matrix size
max_node_id = max(node_features.keys())
feature_size = len(next(iter(node_features.values())))

# Create feature matrix
X = np.zeros((max_node_id + 1, feature_size))
for node_id, features in node_features.items():
    X[node_id] = features

In [6]:
train_edges = []
train_labels = []
with open(train_path , "r") as f:
    for line in f:
        values = line.strip().split()
        node1 = int(values[0])
        node2 = int(values[1])
        label = int(values[2])
        train_edges.append((node1, node2))
        train_labels.append(label)

In [7]:
test_edges = []
with open(test_path , "r") as f:
    for line in f:
        values = line.strip().split()
        node1 = int(values[0])
        node2 = int(values[1])
        test_edges.append((node1, node2))

In [8]:
# Standardize node features
stdscaler = StandardScaler()
node_features = stdscaler.fit_transform(X)

# Graph Building

In [9]:
def build_graph(edges, max_node_id):
    G = nx.Graph() # undirected graph
    G.add_nodes_from(range(max_node_id + 1)) # Add all nodes from 0 to max_node_id (including isolated nodes)
    for edge in edges: # Add edges
        u, v = edge
        G.add_edge(u, v)
    return G

In [10]:
# Build training graph using only positive edges
pos_edges = [edge for edge, label in zip(train_edges, train_labels) if label == 1]
train_graph = build_graph(pos_edges, max_node_id)

# Feature Engineering

In [11]:
def extract_features(edges, graph, node_features, use_topology=True):
    features_list = []

    for u, v in tqdm(edges, desc="Extracting features"):
        # Node features computation
        feat_u = node_features[u]
        feat_v = node_features[v]
        
        # Features Operations
        diff = np.abs(feat_u - feat_v)  # absolute difference: emphasize the difference between nodes
        mult = feat_u * feat_v          # element-wise product: emphasize common important features
        
        # dimensionality reduction
        combined_features = np.concatenate([
            diff,   # absolute difference
            mult    # element-wise product
        ])
        
        # topology feature（Graph structure feature）(designed to capture the structural relationships between nodes in the graph)
        if use_topology and u in graph and v in graph:
            # Common neighbors: Nodes with more common neighbors are more likely to be connected
            cn = len(list(nx.common_neighbors(graph, u, v)))
            

            # Jaccard coefficient:  Measures the similarity of the neighbors of the two nodes
            neighbors_u = set(graph.neighbors(u))
            neighbors_v = set(graph.neighbors(v))
            union_size = len(neighbors_u.union(neighbors_v))
            jaccard = cn / union_size if union_size > 0 else 0
            
            # Preferential attachment: Nodes with higher degrees are more likely to be connected
            pa = graph.degree(u) * graph.degree(v)
            
            # Adamic-Adar index: Gives more weight to low-degree common neighbors when computing similarity
            aa = sum(1 / np.log(graph.degree(w)) for w in nx.common_neighbors(graph, u, v) if graph.degree(w) > 1)
            
            # Resource allocation index: Assumes resource is allocated based on node degrees
            ra = sum(1 / graph.degree(w) for w in nx.common_neighbors(graph, u, v) if graph.degree(w) > 0)
            
            # Node degree: Captures the "activeness" or centrality of the nodes
            degree_u = graph.degree(u)
            degree_v = graph.degree(v)
            
            topo_features = np.array([
                cn, jaccard, pa, aa, ra, degree_u, degree_v,
            ])
            
            # Combine all features
            combined_features = np.concatenate([combined_features, topo_features])
        
        features_list.append(combined_features)
    
    return np.array(features_list)

In [12]:
# Extract features for training data
train_features = extract_features(train_edges, train_graph, node_features)

Extracting features: 100%|██████████| 10496/10496 [00:00<00:00, 54370.77it/s]


# Random Forest Model

In [13]:
# Use Stratified k-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

if DO_HYPERPARAMETER_TUNING:
    # Use Stratified k-fold cross-validation

    # parameter grid for Random Forest
    param_dist = {
        'n_estimators': [100, 200, 300, 400],
        'max_depth': [10, 15, 20, None],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4, 8],
        'max_features': ['sqrt', 'log2', None],
        'class_weight': ['balanced', 'balanced_subsample', None]
    }

    base_model = RandomForestClassifier(random_state=42, n_jobs=-1)
    scorer = make_scorer(roc_auc_score)

    # Randomized search
    random_search = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_dist,
        n_iter=20,
        scoring=scorer,
        cv=cv,
        verbose=1,
        random_state=42,
        n_jobs=-1
    )

    # Find the best hyperparameters
    random_search.fit(train_features, train_labels)

    best_params = random_search.best_params_
    print(f"Best Parameters: {best_params}")
    
    # Use the best hyperparameters to train the model
    model = RandomForestClassifier(
        **best_params,
        random_state=42,
        n_jobs=-1,
        oob_score=True
    )
else:
    # Use default hyperparameters
    param_dist = {
        'n_estimators': 200,
        'max_depth': 15,
        'min_samples_split': 5,
        'min_samples_leaf': 2,
        'max_features': 'sqrt',
        'class_weight': 'balanced'
    }

    model = RandomForestClassifier(
        **param_dist,
        random_state=42,
        n_jobs=-1,
        oob_score=True
    )
    print(f"Parameters: {param_dist}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 10, 'class_weight': None}


# Validation

In [14]:
# cross-validation
cv_scores = cross_val_score(
    model,
    train_features,
    train_labels,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1
)
print(f"Cross-Validation AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f'Each Fold AUC: {cv_scores}')

Cross-Validation AUC: 0.8356 ± 0.0088
Each Fold AUC: [0.83999637 0.83799355 0.8192864  0.83543692 0.84545826]


In [15]:
# Use all training data to train the model
model.fit(train_features, train_labels)

# Check the OOB error rate
oob_error = 1 - model.oob_score_
print(f"OOB Error Rate: {oob_error:.4f}")


OOB Error Rate: 0.2448


# Prediction

In [16]:
# Extract features for test data
test_features = extract_features(test_edges, train_graph, node_features)

test_probs = model.predict_proba(test_features)[:, 1]
test_preds = (test_probs >= 0.5).astype(int)

Extracting features: 100%|██████████| 3498/3498 [00:00<00:00, 35736.90it/s]


In [17]:
# set output file
output_file = os.path.join(base_path, "rf_predictions.csv")

# Output predictions
with open(output_file, "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["ID", "Predicted"])
    for i, pred in enumerate(test_preds):
        writer.writerow([i, int(pred)])

print(f"Predictions saved to {output_file}")

Predictions saved to /Users/cck/Desktop/OneDrive_France/DSBA/T2/Machine Learning in Network Science/Kaggle/code/rf_predictions.csv
